In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as stats
from Basic_preprocessing_copy import basic_preprocessing
import sklearn
import time
from plotly.subplots import make_subplots
import plotly.graph_objs as go


importing Jupyter notebook from Basic_preprocessing_copy.ipynb


### Data importing and preprocessing

이전에 작성한 클러스터링 결과 데이터 importing

클러스터 별 게임 정보 summary

In [2]:
game_result_clustering = pd.read_csv('challenger_game_list_deck.csv')
unit_list = pd.read_csv('challanger_game_unit_list.csv')

game_result_clustering = game_result_clustering.drop('Unnamed: 0', axis=1)

unit_list = unit_list.drop('Unnamed: 0', axis=1)

In [3]:
working = basic_preprocessing(game_result_clustering, unit_list, cluster_on=True)

working.change_gloves_item()
working.check_item_unit_itemver()



,Datetime,Game_id,Player_id,Placement,Deck,augment1,augment2,augment3,champ_name,rarity,tier,item1,item2,item3,num_item
0,2023-03-08,KR_6396160748,9UFWUk8TsO4vXlLduA_UL40ucgwDys9sBPksfqR_4o_jR8...,3.0,우세 리롤,아이템 꾸러미 I,원칙의 원형낫 II,끔찍한 기억,갱플랭크,0.0,2.0,None,None,None,0
1,2023-03-08,KR_6396160748,9UFWUk8TsO4vXlLduA_UL40ucgwDys9sBPksfqR_4o_jR8...,3.0,우세 리롤,아이템 꾸러미 I,원칙의 원형낫 II,끔찍한 기억,바이,1.0,3.0,수호자의 맹세,이온 충격기,덤불 조끼,3
2,2023-03-08,KR_6396160748,9UFWUk8TsO4vXlLduA_UL40ucgwDys9sBPksfqR_4o_jR8...,3.0,우세 리롤,아이템 꾸러미 I,원칙의 원형낫 II,끔찍한 기억,리 신,1.0,3.0,None,None,None,0
3,2023-03-08,KR_6396160748,9UFWUk8TsO4vXlLduA_UL40ucgwDys9sBPksfqR_4o_jR8...,3.0,우세 리롤,아이템 꾸러미 I,원칙의 원형낫 II,끔찍한 기억,말파이트,1.0,3.0,즈롯 차원문,구원,태양불꽃 망토,3
4,2023-03-08,KR_6396160748,9UFWUk8TsO4vXlLduA_UL40ucgwDys9sBPksfqR_4o_jR8...,3.0,우세 리롤,아이템 꾸러미 I,원칙의 원형낫 II,끔찍한 기억,이즈리얼,1.0,3.0,푸른 파수꾼,마법공학 총검,구인수의 격노검,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651611,2023-02-10,KR_6354865470,wtbyNeXaRM8JrtH6tNPanyTyhbcmWeKe3QwvsLgeoKrI80...,6.0,자동방어체계 선의,사냥의 전율 I,최후의 저항,업그레이드: 광란,소나,2.0,2.0,푸른 파수꾼,None,None,1
1651612,2023-02-10,KR_6354865470,wtbyNeXaRM8JrtH6tNPanyTyhbcmWeKe3QwvsLgeoKrI80...,6.0,자동방어체계 선의,사냥의 전율 I,최후의 저항,업그레이드: 광란,르블랑,2.0,2.0,구인수의 격노검,None,None,1
1651613,2023-02-10,KR_6354865470,wtbyNeXaRM8JrtH6tNPanyTyhbcmWeKe3QwvsLgeoKrI80...,6.0,자동방어체계 선의,사냥의 전율 I,최후의 저항,업그레이드: 광란,소라카,4.0,2.0,구인수의 격노검,푸른 파수꾼,None,2
1651614,2023-02-10,KR_6354865470,wtbyNeXaRM8JrtH6tNPanyTyhbcmWeKe3QwvsLgeoKrI80...,6.0,자동방어체계 선의,사냥의 전율 I,최후의 저항,업그레이드: 광란,누누와 윌럼프,6.0,1.0,None,None,None,0


In [4]:
a = time.time()

deck_list = set(working.after_join.Deck)

deck_result = {i : working.making_deck_dictionary(i) for i in deck_list}

b = time.time()

b-a

215.56808710098267

### 덱 정보 요약 함수

일자별 해당 덱 summary, 많이 사용한 증강체, 아이템, 챔피언 summary해주는 함수

In [8]:
def make_deck_visualization(deck_result, deck_name):
    
    deck_summary = deck_result[deck_name]
    
    summary = pd.DataFrame(deck_summary['summary'])

    summary = summary.loc[np.logical_and(summary.Datetime>='2023-01-27', summary.Datetime<='2023-03-14')]


    result_summary = summary.describe()
    
    fig = make_subplots(rows=2, cols=2, subplot_titles = ('일자별 사용 수', '일자별 평균 등수', '일자별 순방률', '일자별 승률'))

    fig.add_trace(go.Scatter(

        x = summary.Datetime,
        y = summary.Count,
        name = 'Count',
        mode = 'lines'
    )
    )

    fig.add_trace(go.Scatter(

        x = summary.Datetime,
        y=summary.Avg_placement,
        name = 'Placement',
        mode = 'lines'

    ), row=1, col=2)

    fig.add_trace(go.Scatter(

        x = summary.Datetime,
        y=summary.Avg_save,
        name = 'Save',
        mode = 'lines'

    ), row=2, col=1)

    fig.add_trace(go.Scatter(

        x = summary.Datetime,
        y=summary.Avg_winner,
        name = 'Winner',
        mode = 'lines'

    ), row=2, col=2)


    fig.add_vline(x = '2023-02-08', line_dash = 'dash')


    fig.add_vline(x = '2023-02-23', line_dash = 'dash')

    fig.add_vline(x = '2023-03-08', line_dash = 'dash')

    fig.add_hline(y = np.mean(summary.Count), line_dash = 'dash', line_color = 'gray', row=1, col=1)

    fig.add_hline(y = np.mean(summary.Avg_placement), line_dash = 'dash', line_color = 'gray', row=1, col=2)

    fig.add_hline(y = np.mean(summary.Avg_save), line_dash = 'dash', line_color = 'gray', row=2, col=1)

    fig.add_hline(y = np.mean(summary.Avg_winner), line_dash = 'dash', line_color = 'gray', row=2, col=2)
    
    fig.update_layout(title = deck_name + ' summary')
    

    deck_augment = pd.DataFrame(deck_summary['augment'])

    deck_top10_aug = deck_augment.sort_values('Count', ascending=False).reset_index(drop=True).iloc[:10, ]


    deck_item = pd.DataFrame(deck_summary['item'])

    deck_top10_item = deck_item.sort_values('Count', ascending=False).reset_index(drop=True).iloc[:10, ]

    
    deck_champ = pd.DataFrame(deck_summary['champion'])

    deck_top10 = deck_champ.sort_values('Count', ascending=False).reset_index(drop=True).iloc[:10, ]
    
    
    
    fig1 = make_subplots(rows=3, cols=2, subplot_titles = ('평균 순위', '순방시 평균 순위', '순방률', '승률', '착용 아이템 수', '키유닛 비율')                  
                   )


    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Ave_score,

        marker_color = px.colors.qualitative.Plotly[:10]


    ))

    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Ave_score_save,

        marker_color = px.colors.qualitative.Plotly[:10]


    ), row=1, col=2)

    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Save_rate,

        marker_color = px.colors.qualitative.Plotly[:10]


    ), row=2, col=1)

    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Win_rate,

        marker_color = px.colors.qualitative.Plotly[:10]


    ), row=2, col=2)

    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Num_items_avg,

        marker_color = px.colors.qualitative.Plotly[:10]


    ), row=3, col=1)


    fig1.add_trace(go.Bar(

        x = deck_top10.Name,
        y = deck_top10.Full_item_rate,

        marker_color = px.colors.qualitative.Plotly[:10]


    ), row=3, col=2)

    fig1.update_layout(showlegend=False, 
        autosize=False,
        width=800,
        height=800,
        title = deck_name + ' champion')
    
    
    
    return result_summary, fig, deck_top10_aug, deck_top10_item, deck_top10, fig1

In [6]:
visualization_summary = {i : make_deck_visualization(deck_result, i) for i in deck_list}

dict_keys(['주문투척자', '무법자 황소부대', '메카 고벨류', '싸움꾼', '결투가', '메카 특등사수', '레이저단', '기계유망주 선의', '우세 리롤', '방패대 고벨류', '동물특공대', '자동방어체계 선의', '위협 정찰단'])

### 덱별 비교하기


그래프의 대시 라인은 패치 적용 날짜

In [66]:
result = (dict(Deck = [], Avg_count = [], Avg_placement = [],Avg_save = [], Avg_winner = []))


for i in visualization_summary.keys():
    
    check = visualization_summary[i][0]
    
    result['Deck'].append(i)
    result['Avg_count'].append(check.loc['mean', 'Count'])
    result['Avg_placement'].append(check.loc['mean', 'Avg_placement'])
    result['Avg_save'].append(check.loc['mean', 'Avg_save'])
    result['Avg_winner'].append(check.loc['mean', 'Avg_winner'])
    
result = pd.DataFrame(result)
    

In [67]:
summary_figure = make_subplots(rows = 2, cols = 2, subplot_titles = ('일자별 평균 사용 횟수', '평균 순위', '순방률', '승률'))

summary_figure.add_trace(go.Bar(
    
    x = result.Deck,
    y = result.Avg_count,
    marker_color = 'skyblue'



), row=1, col=1)


summary_figure.add_trace(go.Bar(
    
    x = result.Deck,
    y = result.Avg_placement,
    marker_color = 'skyblue'



), row=1, col=2)


summary_figure.add_trace(go.Bar(
    
    x = result.Deck,
    y = result.Avg_save,
    marker_color = 'skyblue'



), row=2, col=1)


summary_figure.add_trace(go.Bar(
    
    x = result.Deck,
    y = result.Avg_winner,
    marker_color = 'skyblue'



), row=2, col=2)

summary_figure.update_layout(
    showlegend=False, 
    autosize=False,
    width=1000,
    height=800,
    title = 'Deck별 비교'
    


)

summary_figure.update_layout(
    
    xaxis={'categoryorder': 'total descending'},
    xaxis2={'categoryorder': 'total ascending'},
    xaxis3= {'categoryorder': 'total descending'},
    xaxis4={'categoryorder': 'total descending'}


)
summary_figure

In [68]:
day_deck_summary = pd.DataFrame(dict(Datetime = [], Count = [], Avg_placement = [], Avg_tier = [], Avg_save = [], Avg_winner = [], Deck = []))


for i in deck_list:
    
    check = pd.DataFrame(deck_result[i]['summary'])
    check['Deck'] = i
    
    day_deck_summary = pd.concat([day_deck_summary, check])



In [69]:
day_count = px.line(data_frame = day_deck_summary, x = day_deck_summary.Datetime, y = day_deck_summary.Count, color = day_deck_summary.Deck)

day_count.update_layout(title = '덱 사용 횟수')
day_count.add_vline(x = '2023-02-08', line_dash = 'dash')
day_count.add_vline(x = '2023-02-23', line_dash = 'dash')
day_count.add_vline(x = '2023-03-08', line_dash = 'dash')

In [70]:
day_placement = px.line(data_frame = day_deck_summary, x = day_deck_summary.Datetime, y = day_deck_summary.Avg_placement, color = day_deck_summary.Deck)

day_placement.update_layout(title = '평균 순위')
day_placement.add_vline(x = '2023-02-08', line_dash = 'dash')
day_placement.add_vline(x = '2023-02-23', line_dash = 'dash')
day_placement.add_vline(x = '2023-03-08', line_dash = 'dash')

In [71]:
day_save = px.line(data_frame = day_deck_summary, x = day_deck_summary.Datetime, y = day_deck_summary.Avg_save, color = day_deck_summary.Deck)

day_save.update_layout(title = '순방률')
day_save.add_vline(x = '2023-02-08', line_dash = 'dash')
day_save.add_vline(x = '2023-02-23', line_dash = 'dash')
day_save.add_vline(x = '2023-03-08', line_dash = 'dash')

In [72]:
day_winner = px.line(data_frame = day_deck_summary, x = day_deck_summary.Datetime, y = day_deck_summary.Avg_winner, color = day_deck_summary.Deck)

day_winner.update_layout(title = '승률')
day_winner.add_vline(x = '2023-02-08', line_dash = 'dash')
day_winner.add_vline(x = '2023-02-23', line_dash = 'dash')
day_winner.add_vline(x = '2023-03-08', line_dash = 'dash')

# 덱 별 특징


1. summary - 사용 수와 평균 순위와 순방률, 승률

2. champion - 해당 덱에서 가장 많이 사용한 유닛 리스트, 순방 리스트, 승률 리스트

3. augment - 해당 덱에서 가장 많이 사용한 증강체 리스트

4. item - 해당 덱에서 가장 많이 사용한 아이템

In [36]:
deck_list

{'결투가',
 '기계유망주 선의',
 '동물특공대',
 '레이저단',
 '메카 고벨류',
 '메카 특등사수',
 '무법자 황소부대',
 '방패대 고벨류',
 '싸움꾼',
 '우세 리롤',
 '위협 정찰단',
 '자동방어체계 선의',
 '주문투척자'}

### 결투가

#### summary

사용 수 : 70510

평균 순위 : 4.41

평균 순방률 : 52.9%

평균 승률 : 6.8%


시간별 트렌드

1) 사용 수 : 2월 8일 이후 증가

2) 성적 : 순방률 및 평균 순위는 시간이 지날수록 상승, 승률은 변함없음

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 결투가 문장, 결투가 심장

2) 영웅 증강 : 약자 멸시, 확산 사격, 황소의 활력, 수금, 승리의 장막

3) 이 외 증강 : 사냥의 전율, 나이프의 날, 재료 꾸러미

4) 성적 좋은 3개 증강 : 황소의 활력, 나이프의 날, 결투가 심장

5) 특이사항 : 

#### 아이템

1) 딜러 아이템 : 최후의 속삭임, 밤의 끝자락, 거인 학살자, 무한의 대검, 정의의 손길, 피바라기

2) 탱커 아이템 : 강철의 솔라리 펜던트, 구원

3) 기타 아이템 : 결투가 상징

4) 성적 좋은 아이템 : 대부분의 아이템이 순방률이 55~58%대 유지

5) 특이사항 : 결투가 상징 획득 시 승률 증가

#### 챔피언

1) 키 챔피언 : 벨베스, 아펠리오스, 제드, 베인

2) 특이사항 : 벨베스, 아펠리오스의 경우 결투가의 상징 추가한 경우가 많음.
              4코스트 챔피언의 경우 성적이 비슷함. 의외로 피오라 사용시의 승률이 좋은 편.

In [75]:
visualization_summary['결투가'][1]

### 기계유망주 선의

#### summary

사용 수 : 153222

평균 순위 : 4.49

평균 순방률 : 50.6%

평균 승률 : 9.9%


시간별 트렌드

1) 사용 수 : 2월 8일 이후 사용량 증가

2) 성적 : 2월 8일 이후 성적 증가 but 승률은 감소

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 선의 문장, 고대의 기록 보관소, 기계유망주 심장

2) 영웅 증강 : 이중 방울, 불타는 영혼, 한숨 잘 시간, 융화

3) 이 외 증강 : 휴대용 대장간, 보석 연꽃, 재료 구러미

4) 성적 좋은 3개 증강 : 이중 방울, 선의 문장, 보석 연꽃

5) 특이사항 : 선의 문장, 보석 연꽃 사용 시 승률 증가

#### 아이템

1) 딜러 아이템 : 푸른 파수꾼, 보석 건틀릿, 거인 학살자, 마법공학 총검, 모렐로노미콘, 쇼진의 창

2) 탱커 아이템 : 이온 충격기, 가고일 돌갑옷

3) 기타 아이템 : 선의 상징, 도적의 장갑

4) 성적 좋은 아이템 : 대부분의 아이템의 순방률 50%를 넘어감.

5) 특이사항 : 선의 상징, 마법공학 총검의 승률이 높음

#### 챔피언

1) 키 챔피언 : 조이, 소라카

2) 특이사항 : 룰루까지 사용하는 경우 - 5기계유망주 순위가 높은 것을 확인

In [76]:
visualization_summary['기계유망주 선의'][1]

### 동물특공대

#### summary

사용 수 : 55152

평균 순위 : 4.5

평균 순방률 : 49.3%

평균 승률 : 10.5%


시간별 트렌드

1) 사용 수 : 2월 23일 이후 사용 수 증가

2) 성적 : 승률 제외 성적이 좋아짐

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 동물특공대 문장, 동물특공대 심장, 고대의 기록 보관소

2) 영웅 증강 : 전부 터져라!, 신난다!, 총알은 비를 타고, 토끼 용병

3) 이 외 증강 : 보석 연꽃, 휴대용 대장간, 재료 꾸러미

4) 성적 좋은 3개 증강 : 동물특공대 심장, 토끼 용병, 보석 연꽃

5) 특이사항 : 총알은 비를 타고 사용시 승률이 가장 높음.

#### 아이템

1) 딜러 아이템 : 쇼진의 창, 보석 건틀릿, 거인 학살자, 스태틱의 단검

2) 탱커 아이템 : 태양불꽃 망토, 이온 충격기, 용의 발톱, 워모그의 갑옷, 덤불 조끼

3) 기타 아이템 : 동물특공대 상징

4) 성적 좋은 아이템 : 동물특공대 상징, 태양불꽃 망토, 이온 충격기, 스태틱의 단검, 워모그의 갑옷

5) 특이사항

#### 챔피언

1) 키 챔피언 : 미스 포츈, 리븐

2) 특이사항 : 징크스, 베인, 사일러스 유지하는 동물특공대의 성적이 좋음

In [77]:
visualization_summary['동물특공대'][1]

### 레이저단

#### summary

사용 수 : 100697

평균 순위 : 4.2

평균 순방률 : 55.6%

평균 승률 : 10.2%


시간별 트렌드

1) 사용 수 : 점차 감소

2) 성적 : 약간 감소하는 경향

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 레이저단 문장, 레이저단 심장, 고대의 기록 보관소

2) 영웅 증강 : 추방자의 기백, 약자 멸시, 레이저단 진형, 양떼 도륙

3) 이 외 증강 : 사냥의 전율, 휴대용 대장간, 재료 꾸러미

4) 성적 좋은 3개 증강 : 추방자의 기백, 레이저단 심장, 레이저단 문장

5) 특이사항

#### 아이템

1) 딜러 아이템 : 밤의 끝자락, 스태틱의 단검, 피바라기, 정의의 손길, 최후의 속삭임, 마법공학 총검

2) 탱커 아이템 : 이온 충격기, 태양불꽃 망토

3) 기타 아이템 : 도적의 장갑, 레이저단 상징

4) 성적 좋은 아이템 : 레이저단 상징, 태양불꽃 망토, 도적의 장갑

5) 특이사항

#### 챔피언

1) 키 챔피언 : 제드

2) 특이사항 : 레이저단 문장을 이용, 고코스트 포함 레이저단 구성이 중요함.

In [78]:
visualization_summary['레이저단'][1]

### 메카 고벨류

#### summary

사용 수 : 264498

평균 순위 : 4.1

평균 순방률 : 56.2%

평균 승률 : 19.1%


시간별 트렌드

1) 사용 수 : 점차 증가하다가 감소

2) 성적 : 평균에서 변화 없음

3) 특이사항 : 




#### 증강체

1) 특성 관련 증강

2) 영웅 증강 : 회피, 무모한 자

3) 이 외 증강 : 휴대용 대장간, 초월, 사냥의 전율, 재생의 바람, 보석 연꽃, 재료 꾸러미, 태양불꽃판, 천상의 축복

4) 성적 좋은 3개 증강 : 초월, 무모한 자, 보석 연꽃

5) 특이사항

#### 아이템

1) 딜러 아이템 : 최후의 속삭임, 마법공학 총검, 거인 학살자, 무한의 대검, 피바라기

2) 탱커 아이템 : 구원, 용의 발톱, 덤불 조끼, 태양불꽃 망토, 가고일 돌갑옷

3) 기타 아이템

4) 성적 좋은 아이템 : 거인학살자, 마법공학 총검, 무한의 대검, 피바라기

5) 특이사항

#### 챔피언

1) 키 챔피언 :세트, 아펠리오스, 사미라, 레오나, 벨베스

2) 특이사항 : 메카 유닛 중 세트 or 레오나를 베이스로 진행하되, 딜러를 벨베스 기반 또는 사미라 기반으로 나눌 수 있음, 이 중 사미라 기반 덱이 강함

In [79]:
visualization_summary['메카 고벨류'][1]

### 메카 특등사수

#### summary

사용 수 : 128708

평균 순위 : 4.9

평균 순방률 : 42.2%

평균 승률 : 10.4%


시간별 트렌드

1) 사용 수 : 증가하다 감소

2) 성적 : 점차 낮아짐

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 특등사수 문장

2) 영웅 증강 : 끝없는 피자, 회피, 민간인 문장

3) 이 외 증강 : 초월, 휴대용 대장간, 재생의 바람, 사냥의 전율, 재료 꾸러미, 보석 연꽃

4) 성적 좋은 3개 증강 : 특등사수 문장, 끝없는 피자, 초월

5) 특이사항

#### 아이템

1) 딜러 아이템 : 최후의 속삭임, 마법공학 총검, 거인 학살자, 무한의 대검, 구인수의 격노검

2) 탱커 아이템 : 구원, 용의 발톱, 태양불꽃 망토, 덤불 조끼, 가고일 돌갑옷

3) 기타 아이템

4) 성적 좋은 아이템 : 태양불꽃 망토, 무한의 대검, 거인 학살자

5) 특이사항

#### 챔피언

1) 키 챔피언 : 사미라, 세트 ,아펠리오스, 벨베스

2) 특이사항 : 시비르 사용 시 성적이 좋아짐 - 증강체와의 캐미

In [80]:
visualization_summary['메카 특등사수'][1]

### 무법자 황소부대

#### summary

사용 수 : 152904

평균 순위 : 4.3

평균 순방률 : 52.5%

평균 승률 : 13.8%


시간별 트렌드

1) 사용 수 : 증가하다 감소

2) 성적 : 평균에서 유지

3) 특이사항 : 승률의 경우 시간이 지날수록 약간 감소




#### 증강체

1) 특성 관련 증강 : 황소부대 문장, 무법자 문장, 고대의 기록 보관소

2) 영웅 증강 : 공범, 마법공학 응징, 불타는 영혼

3) 이 외 증강 : 저눝 마법사, 사냥의 전율, 재료 꾸러미

4) 성적 좋은 3개 증강 : 황소부대 문장, 저너투 마법사, 마법공학 응징

5) 특이사항 

#### 아이템

1) 딜러 아이템 : 이온 충격기, 보석 건틀릿, 거인의 결의, 마법공학 총검, 피바라기, 거인 학살자

2) 탱커 아이템 : 태양불꽃 망토

3) 기타 아이템 : 황소부대 상징

4) 성적 좋은 아이템 : 마법공학 총검, 황소부대 상징, 정의의 손길

5) 특이사항

#### 챔피언

1) 키 챔피언 : 비에고, 벨베스, 탈론

2) 특이사항 : 벨베스에 황소부대 사용

In [81]:
visualization_summary['무법자 황소부대'][1]

### 방패대 고벨류

#### summary

사용 수 : 241903

평균 순위 : 4.2

평균 순방률 : 54.3%

평균 승률 : 15.9%


시간별 트렌드

1) 사용 수 : 증가했다 감소

2) 성적 : 아주 미세하게 감소

3) 특이사항




#### 증강체

1) 특성 관련 증강

2) 영웅 증강 : 권투 교습

3) 이 외 증강 : 휴대용 대장간, 보석 연꽃, 사냥의 전율, 초월, 재료 꾸러미, 재생의 바람, 조준경 부착, 천상의 축복, 태양불꽃판

4) 성적 좋은 3개 증강 : 권투 교습, 보석 연꽃, 태양불꽃판

5) 특이사항

#### 아이템

1) 딜러 아이템 : 최후의 속삭임, 마법공학 총검, 거인 학살자, 보석 건틀릿, 쇼진의 창

2) 탱커 아이템 : 구원, 태양불꽃 망토, 용의 발톱, 덤불 조끼

3) 기타 아이템 : 도적의 장갑

4) 성적 좋은 아이템 : 거인 학살자, 도적의 장갑

5) 특이사항

#### 챔피언

1) 키 챔피언 : 벨베스, 사미라, 아펠리오스

2) 특이사항 : 벨베스 기반보다 사미라 기반이 성적이 좋음

In [82]:
visualization_summary['방패대 고벨류'][1]

### 싸움꾼

#### summary

사용 수 : 54668

평균 순위 : 4.78

평균 순방률 : 44.4%

평균 승률 : 7.8%


시간별 트렌드

1) 사용 수 : 점차 증가

2) 성적 : 3월 8일 패치 이후 확 좋아짐

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 싸움꾼 문장, 싸움꾼 심장

2) 영웅 증강 : 가차없는 맹공, 회피, 분노의 지배

3) 이 외 증강 : 초월, 조준경 부착, 보석 연꽃, 재생의 바람, 전투 마법사

4) 성적 좋은 3개 증강 : 조준경 부착, 전투 마법사, 초월, 분노의 지배

5) 특이사항

#### 아이템

1) 딜러 아이템 : 보석 건틀릿, 고석 연사포, 마법공학 총검, 거인 학살자, 피바라기, 푸른 파수꾼

2) 탱커 아이템 : 태양불꽃 망토, 구원, 이온 충격기

3) 기타 아이템 : 싸움꾼 상징

4) 성적 좋은 아이템 : 이온 충격기, 거인 학살자, 태양불꽃 망토

5) 특이사항

#### 챔피언

1) 키 챔피언 : 잭스, 벨베스, 소라카, 레넥톤

2) 특이사항 : 레넥톤 캐리 덱으로 갔을 때 성적이 좋음.

In [83]:
visualization_summary['싸움꾼'][1]

### 우세 리롤

#### summary

사용 수 : 101990

평균 순위 : 4.5

평균 순방률 : 50.6%

평균 승률 : 13.8%


시간별 트렌드

1) 사용 수 : 점차 증가

2) 성적 : 패치에 따라 증가 감소를 병행

3) 특이사항




#### 증강체

1) 특성 관련 증강 

2) 영웅 증강 : 끓어오르는 주문의 힘, 포식자의 정확성, 드레이븐의 리그, 무자비한 칼날

3) 이 외 증강 : 사냥의 전율, 보석 연꽃, 재료 꾸러미, 초월, 휴대용 대장간, 전투 마법사

4) 성적 좋은 3개 증강 : 무자비한 칼날, 포식자의 정확성, 드레이븐의 리그, 전투 마법사

5) 특이사항 - 키 유닛 영웅 증강 존재시 성적이 엄청 증가함

#### 아이템

1) 딜러 아이템 : 푸른 파수꾼, 거인 학살자, 보석 건틀릿, 마법공학 총검, 거인의 결의, 최후의 속삭임

2) 탱커 아이템 : 구원, 태양불꽃 망토, 이온 충격기

3) 기타 아이템 : 도적의 장갑

4) 성적 좋은 아이템 :  : 도적의 장갑, 구우너, 이온 충격기

5) 특이사항

#### 챔피언

1) 키 챔피언 : 유미, 드레이븐, 이즈리얼, 닐라, 바이

2) 특이사항 - 우세 챔피언 기반, 키 유닛을 기준으로 덱이 분류 됨 - 우세 이즈리얼, 우세 유미, 우세 드레이븐

In [84]:
visualization_summary['우세 리롤'][1]

### 위협 정찰단

#### summary

사용 수 : 115092

평균 순위 : 4.42

평균 순방률 : 50.5%

평균 승률 : 10.8%


시간별 트렌드

1) 사용 수 : 증가 후 감소

2) 성적 : 큰 변화 없음

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 정찰단 문장, 정찰단 심장

2) 영웅 증강 : 도굴꾼의 전리품, 다중 사격, 확산 사격

3) 이 외 증강 : 보석 연꽃, 재료 꾸러미, 휴대용 대장간, 사냥의 전율, 재생의 바람

4) 성적 좋은 3개 증강 : 확산 사격, 도굴꾼의 전리품, 휴대용 대장간

5) 특이사항

#### 아이템

1) 딜러 아이템 : 마법공학 총검, 스태틱의 단검, 구인수의 격노검, 거인 학살자

2) 탱커 아이템 : 태양불꽃 망토, 용의 발톱, 구원, 덤불 조끼, 가고일 돌갑옷

3) 기타 아이템 : 도적의 장갑

4) 성적 좋은 아이템 : 도적의 장갑, 태양불꽃 망토

5) 특이사항

#### 챔피언

1) 키 챔피언 : 카이사, 벨베스, 람머스, 베인

2) 특이사항 : 벨베스에 정찰단 문장 사용

In [85]:
visualization_summary['위협 정찰단'][1]

### 자동방어체계 선의

#### summary

사용 수 : 144711

평균 순위 : 4.44

평균 순방률 : 51.0%

평균 승률 : 15.2%


시간별 트렌드

1) 사용 수 : 시간이 지날수록 증가

2) 성적 : 점차 성적 좋아짐

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 자동방어체계 문장, 고대의 기록 보관소, 자동방어체계 심장

2) 영웅 증강 : 정화의 방호, 조준 보정, 융화, 업그레이드 : 광란, 마법공학 응징

3) 이 외 증강 : 보석 연꽃, 허수아비 전선

4) 성적 좋은 3개 증강 : 정화의 방호, 보석 연꽃, 허수아비 전선, 마법공학 응징

5) 특이사항

#### 아이템

1) 딜러 아이템 : 푸른 파수꾼, 보석 건틀릿, 거인 학살자, 마법공학 총검, 쇼진의 창

2) 탱커 아이템 : 태양불꽃 망토, 덤불 조끼, 가고일 돌갑옷

3) 기타 아이템 : 자동방어체계 문장

4) 성적 좋은 아이템 : 이온 충격기, 마법공학 총검, 태양불꽃 망토

5) 특이사항

#### 챔피언

1) 키 챔피언 : 소라카, 르블랑, 에코, 리신

2) 특이사항

In [86]:
visualization_summary['자동방어체계 선의'][1]

### 주문투척자

#### summary

사용 수 : 67561

평균 순위 : 4.46

평균 순방률 : 51.9%

평균 승률 : 9.5%


시간별 트렌드

1) 사용 수 : 2월 23일 이후 대폭 증가

2) 성적 : 2월 8일 이후 성적 좋아짐

3) 특이사항




#### 증강체

1) 특성 관련 증강 : 주문투척자 문장, 별 수호자 문장, 주문투척자 심장, 고대의 기록 보관소

2) 영웅 증강 : 불타는 영혼, 돌처럼 단단하게, 바위술사

3) 이 외 증강 : 보석 연꽃, 재료 꾸러미, 휴대용 대장간

4) 성적 좋은 3개 증강 : 주문투척자 문장, 보석 연꽃, 재료 꾸러미

5) 특이사항

#### 아이템

1) 딜러 아이템 : 쇼진의 창, 보석 건틀릿, 거인 학살자, 스태틱의 단검, 푸른 파수꾼, 마법공학 총검

2) 탱커 아이템 : 이온 충격기, 태양불꽃 망토, 가고일 돌갑옷, 워모그의 갑옷

3) 기타 아이템

4) 성적 좋은 아이템 : 이온 충격기, 태양불꽃 망토, 스태틱의 단검, 마법공학 총검

5) 특이사항

#### 챔피언

1) 키 챔피언 : 탈리야

2) 특이사항

In [87]:
visualization_summary['주문투척자'][1]